In [1]:
!pip install datasets
!pip install transformers 
!pip install evaluate
!pip install rouge-score

In [248]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import datasets
import re
import time
import math

# Read in files and subset

In [205]:
summary_df = pd.read_csv(Path().absolute().parents[1]/Path("modified_data")/"text_and_summaries.csv")
summary_df = summary_df[(summary_df.keep == 1) & (summary_df.summary != 'still need') & ~summary_df.summary.isna()].copy(deep=True)

full_text =  pd.read_csv(Path().absolute().parents[1]/Path("modified_data")/"bill_texts_full_text.csv")
full_text_sample = full_text[full_text.state == "IN"].copy(deep=True)

all_text = summary_df[['state_code', 'state_name', 'bill_name', 'summary']].merge(full_text_sample[['state', 'bill_name', 'text', 'cleaned_text']], left_on = ['state_code', 'bill_name'], right_on = ['state', 'bill_name'])
all_text.rename({'bill_name':'title'}, axis='columns', inplace=True)
all_text.head()

,state_code,state_name,title,summary,state,text,cleaned_text
0,IN,Indiana,HB1589,This bill allows a claimant to bring a medical...,IN,Introduced Version HOUSE BILL No 1589 ...,SECTION IC 18 12 IS ADDED TO THE INDIANA CODE ...
1,IN,Indiana,HB1524,This bill provides that the gender listed on a...,IN,Introduced Version HOUSE BILL No 1524 ...,SECTION IC 37 20 IS ADDED TO THE INDIANA CODE ...
2,IN,Indiana,HB1569,This bill provides that the department of corr...,IN,EH1569 1 March 24 2023 ENGROSSED HOUSE BILL...,SECTION IC 10 1 IS AMENDED TO READ AS FOLLOWS ...
3,IN,Indiana,HB1520,This bill makes it a Class B misdemeanor if a ...,IN,Introduced Version HOUSE BILL No 1520 ...,SECTION IC 31 2 3 IS ADDED TO THE INDIANA CODE...
4,IN,Indiana,HB1231,This bill prohibits specified health care prof...,IN,Introduced Version HOUSE BILL No 1231 ...,SECTION IC 15 23 IS ADDED TO THE INDIANA CODE ...


In [166]:
list(all_text.text)[0]

'  Introduced Version HOUSE BILL No  1589       DIGEST OF  INTRODUCED B ILL Citations Affected   IC 34 18  Synopsis   Malpractice regarding gender therapeutics  Allows a claimant to bring a medical malpractice claim regarding gender transition treatment within 15 years after the individual becomes 21 years of age   Effective   July 1  2023  McGuire January 19  2023  read first time and referred to Commi ttee on Public Health  2023 IN 1589LS 7226 DI 149 Introduced First Reg ular Sessio n of the 123rd General A ssem bly  2023  PRINTING CODE  Amendments  Whenever an existing statute  or a section of the Indiana Constitution  is being amended  the text of the existing provision will appear in this style type  additions will appear in this sty le type  and deletions will appear in this style type    Additions  Whenever a new statutory provision is being enacted  or a new constitutional provision adopted   the text of the new provision will appear in  this  style  type  Also  the word NEW wi

In [167]:
list(all_text.cleaned_text)[0]

'SECTION IC 18 12 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 7 Gender transition means the process of changing an individual s bodily appearance or sex characteristics to a different gender from the gender of the individual at birth by 1 surgery or 2 prescribing administering or dispensing a hormone or drug SECTION IC 18 1 IS AMENDED TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec a This section applies to all persons regardless of minority or other legal disability except as provided in subsection c b Except as provided in subsection d a claim whether in contract or tort may not be brought against a health care provider based upon professional services or health care that was provided or that should have been provided unless the claim is filed within two 2023 IN LS DI 2 years after the date of the alleged act omission or neglect except that a minor less than six years of age has until the minor s eighth birthday to file c Except as provided in 

## Split into training and testing

In [206]:
df = all_text.drop('text', axis='columns')
df.rename({'cleaned_text':'text'}, axis = 'columns', inplace=True)

train, test = train_test_split(all_text[['text', 'summary']], test_size = 0.2)

# Create HuggingFace Objects

In [208]:
train_dataset = datasets.Dataset.from_dict(train)
test_dataset = datasets.Dataset.from_dict(test)
billsum = datasets.DatasetDict({"train": train_dataset,
                                "test": test_dataset
                               })

In [64]:
# from datasets import load_dataset

# Data Loading

In [65]:
# # Load the smaller California state bill subset of the BillSum dataset
# billsum = load_dataset("billsum", split="ca_test")

# # Split into train and test dataset
# billsum = billsum.train_test_split(test_size=0.2)

Found cached dataset billsum (/Users/corrina/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc)
Parameter 'generator'=Generator(PCG64) of the transform datasets.arrow_dataset.Dataset.train_test_split couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [209]:
billsum['test'][0]['summary']

'This bill allows a claimant to bring a medical malpractice claim regarding gender transition treatment within 15 years after the individual becomes 21 years of age.'

In [100]:
# from google.colab import drive
# drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

# Load Tokenizer

In [215]:
## PREPROCESSING
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/corrina/.cache/huggingface/hub/models--t5-small/snapshots/5bf53e1f76b1430d9302d735c613c5f5677e32a6/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "lengt

In [216]:
#The preprocessing function you want to create needs to:

# 1. Prefix the input with a prompt so T5 knows this is a summarization task. 
#    Some models capable of multiple NLP tasks require prompting for specific tasks.
# 2. Use the keyword text_target argument when tokenizing labels.
# 3. Truncate sequences to be no longer than the maximum length set by the max_length parameter.

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    # Tokenizes and truncates
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [223]:
# To apply the preprocessing function over the entire dataset
# You can speed up the map function by setting batched=True to process multiple elements of the dataset at once
tokenized_billsum = billsum.map(preprocess_function, batched=True)
# tokenized_billsum = billsum.map(preprocess_function)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [224]:
tokenized_billsum['train']

Dataset({
    features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14
})

In [225]:
# Create batch of examples using DataCollatorForSeq2Seq. It's better to dynamically pad the sentences to the longest length
# in a batch during collation instead of padding to the model's maximum length
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Set Up Evaluation

In [226]:
## SET UP EVALUATOR

import evaluate
import numpy as np

# Load the ROUGE metric
rouge = evaluate.load("rouge")

# Create function that passes predictions and labels to compute the ROUGE metric
# This is what is used for model training
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Train Model

In [227]:
#model.save_weights("/content/gdrive/My Drive/weights.h5")

In [228]:
# output_directory = "/content/gdrive/My Drive/my_awesome_billsum_model"
output_directory = Path().absolute().parents[1]/Path("modified_data")

In [229]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
#     output_dir="/content/gdrive/My Drive/my_awesome_billsum_model",
    output_dir = output_directory,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
#     fp16=True,
    push_to_hub=False, # changed to false
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



loading configuration file config.json from cache at /Users/corrina/.cache/huggingface/hub/models--t5-small/snapshots/5bf53e1f76b1430d9302d735c613c5f5677e32a6/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size

In [230]:
start = time.time()
trainer.train()
end = time.time()
print(end - start)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, summary. If text, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 14
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4
  Number of trainable parameters = 60506624


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.677066,0.072800,0.000000,0.072800,0.072800,19.000000
2,No log,1.642606,0.072800,0.000000,0.072800,0.072800,19.000000
3,No log,1.617980,0.072800,0.000000,0.072800,0.072800,19.000000
4,No log,1.607086,0.072800,0.000000,0.072800,0.072800,19.000000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, summary. If text, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, summary. If text, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, summary. If text, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Ba

171.72827124595642


In [238]:
180/60

3.0

In [239]:
# Save the model
# trainer.save_model("/content/gdrive/My Drive/my_awesome_billsum_model")
trainer.save_model(output_directory/"summarizer_model")

Saving model checkpoint to /Users/corrina/Documents/anly-521/final_project/ANLY521_Final_Project/modified_data/summarizer_model
Configuration saved in /Users/corrina/Documents/anly-521/final_project/ANLY521_Final_Project/modified_data/summarizer_model/config.json
Model weights saved in /Users/corrina/Documents/anly-521/final_project/ANLY521_Final_Project/modified_data/summarizer_model/pytorch_model.bin
tokenizer config file saved in /Users/corrina/Documents/anly-521/final_project/ANLY521_Final_Project/modified_data/summarizer_model/tokenizer_config.json
Special tokens file saved in /Users/corrina/Documents/anly-521/final_project/ANLY521_Final_Project/modified_data/summarizer_model/special_tokens_map.json


# Test Model

In [254]:
bill_text

'SECTION IC 15 23 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 1 Sec The office may not use federal or state funds under the Medicaid program to pay or provide coverage for gender transition procedures as defined by IC 35 6 to an individual who is less than eighteen years of age SECTION IC 18 36 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 1 Biological sex for purposes of IC 35 has the meaning set forth in IC 35 1 SECTION IC 18 88 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 7 Cross sex hormones for purposes of IC 35 has the meaning set forth in IC 35 2 SECTION IC 18 144 IS ADDED TO THE INDIANA IN LS DI 2 CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 2 Gender for purposes of IC 35 has the meaning set forth in IC 35 3 SECTION IC 18 144 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 3 Gender reassignment su

In [258]:
# text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
bill_text = list(all_text.cleaned_text)[4]
chunk_size = 512
chunks = len(bill_text) // chunk_size
bill_text_parts = [ bill_text[i:i + chunk_size] for i in range(0, len(bill_text), chunk_size)]
bill_text_parts

['SECTION IC 15 23 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 1 Sec The office may not use federal or state funds under the Medicaid program to pay or provide coverage for gender transition procedures as defined by IC 35 6 to an individual who is less than eighteen years of age SECTION IC 18 36 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 1 Biological sex for purposes of IC 35 has the meaning set forth in IC 35 1 SECTION IC 18 88',
 ' IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 7 Cross sex hormones for purposes of IC 35 has the meaning set forth in IC 35 2 SECTION IC 18 144 IS ADDED TO THE INDIANA IN LS DI 2 CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 2 Gender for purposes of IC 35 has the meaning set forth in IC 35 3 SECTION IC 18 144 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 3 Gender reassignm

In [252]:
chunks

26

In [259]:
from transformers import pipeline

summarizer = pipeline("summarization", model=output_directory/"summarizer_model")

for each_bill_part in bill_text_parts:
    print("BILL SUBSET:")
    print("\t", each_bill_part)
    print("BILL SUMMARY:")
    print(summarizer(each_bill_part))
    print("--------------------------------------------------------------------------")
    

loading configuration file /Users/corrina/Documents/anly-521/final_project/ANLY521_Final_Project/modified_data/summarizer_model/config.json
Model config T5Config {
  "_name_or_path": "/Users/corrina/Documents/anly-521/final_project/ANLY521_Final_Project/modified_data/summarizer_model",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_leng

BILL SUBSET:
	 SECTION IC 15 23 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 1 Sec The office may not use federal or state funds under the Medicaid program to pay or provide coverage for gender transition procedures as defined by IC 35 6 to an individual who is less than eighteen years of age SECTION IC 18 36 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 1 Biological sex for purposes of IC 35 has the meaning set forth in IC 35 1 SECTION IC 18 88
BILL SUMMARY:


Your max_length is set to 200, but you input_length is only 188. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=94)


[{'summary_text': 'IC 15 23 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 1 Sec The office may not use federal or state funds under the Medicaid program to pay or provide coverage for gender transition procedures as defined by IC 35 6 .'}]
--------------------------------------------------------------------------
BILL SUBSET:
	  IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 7 Cross sex hormones for purposes of IC 35 has the meaning set forth in IC 35 2 SECTION IC 18 144 IS ADDED TO THE INDIANA IN LS DI 2 CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 2 Gender for purposes of IC 35 has the meaning set forth in IC 35 3 SECTION IC 18 144 IS ADDED TO THE INDIANA CODE AS A NEW SECTION TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec 3 Gender reassignment surgery for purposes of IC 
BILL SUMMARY:


Your max_length is set to 200, but you input_length is only 180. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=90)


...

[{'summary_text': 'ervices or 2 payment or reimbursement for health care services from a health carrier Sec As used in this section gender transition procedures has the meaning set forth in IC 35 6 Sec a Health carrier means an entity that is subject to this title and the administrative rules adopted under this title .'}]
--------------------------------------------------------------------------
BILL SUBSET:
	  care services b The term includes the following 1 An insurer as defined in IC 1 3 x that issues a policy of accident and sickness insurance as defined in IC 8 1 a 2 A health maintenance organization as defined in IN LS DI 8 IC 13 19 3 An administrator as defined in IC 1 1 a that is licensed under IC 1 4 A state employee health plan offered under IC 10 5 A short term insurance plan as defined by IC 8 9 6 Any other entity that provides a plan of health insurance health benefits or health care services c The 
BILL SUMMARY:


Your max_length is set to 200, but you input_length is only 98. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)


[{'summary_text': 'the term includes the following 1 an insurer as defined in IC 1 3 x that issues a policy of accident and sickness insurance . a 2 a health maintenance organization as described in IN LS DI 8 IC 13 19 3 An administrator as defined . in the term . the term is referred to as a short term insurance plan .'}]
--------------------------------------------------------------------------
BILL SUBSET:
	 term does not include 1 an insurer that issues a policy of accident and sickness insurance 2 a limited service health maintenance organization as defined in IC 13 4 or 3 an administrator that only provides coverage for or processes claims for dental or vision care services Sec a A health plan may not include coverage or reimbursement for gender transition procedures for an individual who is less than eighteen years of age b A health plan is not required to provide coverage for gender transition procedures S
BILL SUMMARY:


Your max_length is set to 200, but you input_length is only 69. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


[{'summary_text': 'term does not include 1 an insurer that issues a policy of accident and sickness insurance 2 a limited service health maintenance organization as defined in IC 13 4 or 3 an administrator that only provides coverage for or processes claims for dental or vision care services Sec a A health plan may not include coverage or reimbursement for gender transition procedures for an individual who is less than eighteen years of age .'}]
--------------------------------------------------------------------------
BILL SUBSET:
	 ECTION IC 18 1 IS AMENDED TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec This article does not apply to 1 an act of malpractice that occurred before July 1975 or 2 an action brought under IC 35 gender transition procedures for minors IN LS DI 147
BILL SUMMARY:
[{'summary_text': 'ECTION IC 18 1 IS AMENDED TO READ AS FOLLOWS EFFECTIVE JULY 2023 Sec This article does not apply to 1 an act of malpractice that occurred before July 1975 or 2 an action brought under 

In [235]:
billtext = "An Act concerning education; relating to school districts and employees thereof; requiring parental consent for use of a student's pronouns; prohibiting schools from requiring use of an individual's pronouns over moral or religious objections; requiring school districts to adopt policies thereon. Be it enacted by the Legislature of the State of Kansas: Section 1. (a) An employee or independent contractor of a school district shall not knowingly address, identify or refer to a student who is less than 18 years of age by a pronoun that differs from the pronoun that aligns with the student's biological sex unless the school district has received written permission from the student's parent or guardian. (b) A school district shall not require an employee or independent contractor of such school district to address, identify or refer to an individual by a pronoun that differs from the pronoun that aligns with the student's biological sex if doing so is contrary to the employee's or independent contractor's moral or religious convictions. (c) The school board of each school district shall adopt a policy to implement this section. (d) This section shall not be construed to prohibit any employee or independent contractor of a school district from discussing matters of public concern outside such employee's or independent contractor's official duties. Sec. 2. This act shall take effect and be in force from and after its publication in the statute book"

In [236]:
summarizer(billtext)

[{'summary_text': "an act concerning education; relating to school districts and employees thereof; prohibiting schools from requiring use of an individual's pronouns over moral or religious objections . a school district shall not knowingly address, identify or refer to a student who is less than 18 years of age unless the school district has received written permission from the student's parent or guardian ."}]

In [237]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 14
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 4
    })
})